In [1]:
import pandas as pd

In [17]:
ref = "/mnt/scratch2/avi/meta-map/kraken/KrakenDB/seqid2taxid.map"
with open(ref) as f:
    refId2TaxId = pd.read_csv(f, header=None, sep="\t", names=["refName", "refId"])

In [18]:
refId2TaxId.head()

,refName,refId
0,gi|9791176|ref|NC_002180.1|,1986029
1,gi|10803547|ref|NC_001869.1|,64091
2,gi|10954488|ref|NC_001732.1|,243232
3,gi|10954532|ref|NC_001733.1|,243232
4,gi|10954552|ref|NC_001773.1|,272844


In [24]:
tf = "/mnt/scratch2/avi/meta-map/kraken/KrakenDB/taxonomy/nodes.dmp"
taxa = []
with open(tf) as f:
    for line in f:
        toks = line.rstrip("\t|\n").split("\t|\t")
        taxa += [[int(toks[0]), toks[2]]]

In [30]:
taxa_df = pd.DataFrame.from_records(taxa)
taxa_df.columns = ["taxaId", "rank"]

In [31]:
taxa_df.head()

,taxaId,rank
0,1,no rank
1,2,superkingdom
2,6,genus
3,7,species
4,9,species


In [41]:
ref2taxaLevel = pd.merge(refId2TaxId, taxa_df, left_on="refId", right_on="taxaId")
ref2taxaLevel.head()

,refName,refId,taxaId,rank
0,gi|9791176|ref|NC_002180.1|,1986029,1986029,species
1,gi|10803547|ref|NC_001869.1|,64091,64091,no rank
2,gi|15789340|ref|NC_002607.1|,64091,64091,no rank
3,gi|16119979|ref|NC_002608.1|,64091,64091,no rank
4,gi|10954488|ref|NC_001732.1|,243232,243232,no rank


In [38]:
print(len(refId2TaxId))
print(len(ref2taxaLevel))

4815
4815


# Reference Rank Distribution

In [42]:
ref2taxaLevel.groupby('rank').count()

,refName,refId,taxaId
rank,,,
no rank,4336,4336,4336
species,465,465,465
subspecies,14,14,14
